# Project Title
### Data Engineering Capstone Project

#### Project Summary
--describe your project at a high level--

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [1]:
# Do all imports and installs here
import pandas as pd
import os

from pyspark.sql import SparkSession, DataFrame

### Step 1: Scope the Project and Gather Data

#### Scope 
Explain what you plan to do in the project in more detail. What data do you use? What is your end solution look like? What tools did you use? etc>

#### Describe and Gather Data 
Describe the data sets you're using. Where did it come from? What type of information is included? 

In [4]:
# i94
fname = '../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat'
sampledf = pd.read_sas(fname, 'sas7bdat', encoding="ISO-8859-1")

In [5]:
sampledf.head()

cicid   i94yr  i94mon  i94cit  i94res i94port  arrdate  i94mode i94addr  \
0    6.0  2016.0     4.0   692.0   692.0     XXX  20573.0      NaN     NaN   
1    7.0  2016.0     4.0   254.0   276.0     ATL  20551.0      1.0      AL   
2   15.0  2016.0     4.0   101.0   101.0     WAS  20545.0      1.0      MI   
3   16.0  2016.0     4.0   101.0   101.0     NYC  20545.0      1.0      MA   
4   17.0  2016.0     4.0   101.0   101.0     NYC  20545.0      1.0      MA   

   depdate   ...     entdepu  matflag  biryear   dtaddto gender insnum  \
0      NaN   ...           U      NaN   1979.0  10282016    NaN    NaN   
1      NaN   ...           Y      NaN   1991.0       D/S      M    NaN   
2  20691.0   ...         NaN        M   1961.0  09302016      M    NaN   
3  20567.0   ...         NaN        M   1988.0  09302016    NaN    NaN   
4  20567.0   ...         NaN        M   2012.0  09302016    NaN    NaN   

  airline        admnum  fltno visatype  
0     NaN  1.897628e+09    NaN       B2  
1     NaN  3.736796e+09  00296       F1  
2      OS  6.666432e+08     93       B2  
3      AA  9.246846e+10  00199       B2  
4      AA  9.246846e+10  00199       B2  

[5 rows x 28 columns]

In [6]:
# Sample I94
samplefname = 'immigration_data_sample.csv'
sample_df = pd.read_csv(samplefname)

In [7]:
sample_df.head()

Unnamed: 0      cicid   i94yr  i94mon  i94cit  i94res i94port  arrdate  \
0     2027561  4084316.0  2016.0     4.0   209.0   209.0     HHW  20566.0   
1     2171295  4422636.0  2016.0     4.0   582.0   582.0     MCA  20567.0   
2      589494  1195600.0  2016.0     4.0   148.0   112.0     OGG  20551.0   
3     2631158  5291768.0  2016.0     4.0   297.0   297.0     LOS  20572.0   
4     3032257   985523.0  2016.0     4.0   111.0   111.0     CHM  20550.0   

   i94mode i94addr    ...     entdepu  matflag  biryear   dtaddto  gender  \
0      1.0      HI    ...         NaN        M   1955.0  07202016       F   
1      1.0      TX    ...         NaN        M   1990.0  10222016       M   
2      1.0      FL    ...         NaN        M   1940.0  07052016       M   
3      1.0      CA    ...         NaN        M   1991.0  10272016       M   
4      3.0      NY    ...         NaN        M   1997.0  07042016       F   

  insnum airline        admnum  fltno  visatype  
0    NaN      JL  5.658267e+10  00782        WT  
1    NaN     *GA  9.436200e+10  XBLNG        B2  
2    NaN      LH  5.578047e+10  00464        WT  
3    NaN      QR  9.478970e+10  00739        B2  
4    NaN     NaN  4.232257e+10   LAND        WT  

[5 rows x 29 columns]

In [8]:
# World Temperature Data
temperaturefname = '../../data2/GlobalLandTemperaturesByCity.csv'
temperaturedf = pd.read_csv(temperaturefname)

In [9]:
temperaturedf.head()

dt  AverageTemperature  AverageTemperatureUncertainty   City  \
0  1743-11-01               6.068                          1.737  Århus   
1  1743-12-01                 NaN                            NaN  Århus   
2  1744-01-01                 NaN                            NaN  Århus   
3  1744-02-01                 NaN                            NaN  Århus   
4  1744-03-01                 NaN                            NaN  Århus   

   Country Latitude Longitude  
0  Denmark   57.05N    10.33E  
1  Denmark   57.05N    10.33E  
2  Denmark   57.05N    10.33E  
3  Denmark   57.05N    10.33E  
4  Denmark   57.05N    10.33E

In [10]:
# U.S. City Demographic Data
demographicfname = 'us-cities-demographics.csv'
demographicdf = pd.read_csv(demographicfname, sep=';')

In [11]:
demographicdf.head()

City          State  Median Age  Male Population  \
0     Silver Spring       Maryland        33.8          40601.0   
1            Quincy  Massachusetts        41.0          44129.0   
2            Hoover        Alabama        38.5          38040.0   
3  Rancho Cucamonga     California        34.5          88127.0   
4            Newark     New Jersey        34.6         138040.0   

   Female Population  Total Population  Number of Veterans  Foreign-born  \
0            41862.0             82463              1562.0       30908.0   
1            49500.0             93629              4147.0       32935.0   
2            46799.0             84839              4819.0        8229.0   
3            87105.0            175232              5821.0       33878.0   
4           143873.0            281913              5829.0       86253.0   

   Average Household Size State Code                       Race  Count  
0                    2.60         MD         Hispanic or Latino  25924  
1                    2.39         MA                      White  58723  
2                    2.58         AL                      Asian   4759  
3                    3.18         CA  Black or African-American  24437  
4                    2.73         NJ                      White  76402

In [12]:
airpotcodefname = 'airport-codes_csv.csv'
airpotcodedf = pd.read_csv(airpotcodefname)

In [13]:
airpotcodedf.head()

ident           type                                name  elevation_ft  \
0   00A       heliport                   Total Rf Heliport          11.0   
1  00AA  small_airport                Aero B Ranch Airport        3435.0   
2  00AK  small_airport                        Lowell Field         450.0   
3  00AL  small_airport                        Epps Airpark         820.0   
4  00AR         closed  Newport Hospital & Clinic Heliport         237.0   

  continent iso_country iso_region  municipality gps_code iata_code  \
0       NaN          US      US-PA      Bensalem      00A       NaN   
1       NaN          US      US-KS         Leoti     00AA       NaN   
2       NaN          US      US-AK  Anchor Point     00AK       NaN   
3       NaN          US      US-AL       Harvest     00AL       NaN   
4       NaN          US      US-AR       Newport      NaN       NaN   

  local_code                            coordinates  
0        00A     -74.93360137939453, 40.07080078125  
1       00AA                 -101.473911, 38.704022  
2       00AK            -151.695999146, 59.94919968  
3       00AL  -86.77030181884766, 34.86479949951172  
4        NaN                    -91.254898, 35.6087

In [14]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.\
config("spark.jars.repositories", "https://repos.spark-packages.org/").\
config("spark.jars.packages", "saurfang:spark-sas7bdat:2.0.0-s_2.11").\
enableHiveSupport().getOrCreate()

df_spark = spark.read.format('com.github.saurfang.sas.spark').load('../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat')


In [11]:
df_spark.count()

3096313

In [15]:
df_spark.printSchema()

root
 |-- cicid: double (nullable = true)
 |-- i94yr: double (nullable = true)
 |-- i94mon: double (nullable = true)
 |-- i94cit: double (nullable = true)
 |-- i94res: double (nullable = true)
 |-- i94port: string (nullable = true)
 |-- arrdate: double (nullable = true)
 |-- i94mode: double (nullable = true)
 |-- i94addr: string (nullable = true)
 |-- depdate: double (nullable = true)
 |-- i94bir: double (nullable = true)
 |-- i94visa: double (nullable = true)
 |-- count: double (nullable = true)
 |-- dtadfile: string (nullable = true)
 |-- visapost: string (nullable = true)
 |-- occup: string (nullable = true)
 |-- entdepa: string (nullable = true)
 |-- entdepd: string (nullable = true)
 |-- entdepu: string (nullable = true)
 |-- matflag: string (nullable = true)
 |-- biryear: double (nullable = true)
 |-- dtaddto: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- insnum: string (nullable = true)
 |-- airline: string (nullable = true)
 |-- admnum: double (nullable = 

In [ ]:
#write to parquet
df_spark.write.parquet("sas_data")
df_spark=spark.read.parquet("sas_data")

### Step 2: Explore and Assess the Data
#### Explore the Data 
Identify data quality issues, like missing values, duplicate data, etc.

#### Cleaning Steps
Document steps necessary to clean the data

#### Cleaning the World Temperature Data

In [16]:
# Performing cleaning tasks here
temperaturedf.head()

dt  AverageTemperature  AverageTemperatureUncertainty   City  \
0  1743-11-01               6.068                          1.737  Århus   
1  1743-12-01                 NaN                            NaN  Århus   
2  1744-01-01                 NaN                            NaN  Århus   
3  1744-02-01                 NaN                            NaN  Århus   
4  1744-03-01                 NaN                            NaN  Århus   

   Country Latitude Longitude  
0  Denmark   57.05N    10.33E  
1  Denmark   57.05N    10.33E  
2  Denmark   57.05N    10.33E  
3  Denmark   57.05N    10.33E  
4  Denmark   57.05N    10.33E

In [17]:
temperaturedf.shape

(8599212, 7)

In [18]:
len(temperaturedf['Country'].unique())

159

In [19]:
# Just get temperature of USA
df_temperature = temperaturedf[temperaturedf['Country']=='United States'].copy()

In [20]:
df_temperature.shape

(687289, 7)

In [21]:
#df_temperature['convertDt'] = pd.to_datetime(df_temperature.dt)
df_temperature['convertDt'] = df_temperature['dt']
df_temperature['convertDt'] = pd.to_datetime(df_temperature['convertDt'])

In [22]:
df_temperature.head()

dt  AverageTemperature  AverageTemperatureUncertainty     City  \
47555  1820-01-01               2.101                          3.217  Abilene   
47556  1820-02-01               6.926                          2.853  Abilene   
47557  1820-03-01              10.767                          2.395  Abilene   
47558  1820-04-01              17.989                          2.202  Abilene   
47559  1820-05-01              21.809                          2.036  Abilene   

             Country Latitude Longitude  convertDt  
47555  United States   32.95N   100.53W 1820-01-01  
47556  United States   32.95N   100.53W 1820-02-01  
47557  United States   32.95N   100.53W 1820-03-01  
47558  United States   32.95N   100.53W 1820-04-01  
47559  United States   32.95N   100.53W 1820-05-01

In [23]:
df_temperature.isnull().sum()

dt                                   0
AverageTemperature               25765
AverageTemperatureUncertainty    25765
City                                 0
Country                              0
Latitude                             0
Longitude                            0
convertDt                            0
dtype: int64

In [24]:
# get the data is Null or NaN with AverageTemperature & AverageTemperatureUncertainty
df_temp_null = df_temperature[df_temperature.AverageTemperature.isnull()]
df_temp_null.head()

dt  AverageTemperature  AverageTemperatureUncertainty  \
47723   1834-01-01                 NaN                            NaN   
137067  1743-12-01                 NaN                            NaN   
137068  1744-01-01                 NaN                            NaN   
137069  1744-02-01                 NaN                            NaN   
137070  1744-03-01                 NaN                            NaN   

           City        Country Latitude Longitude  convertDt  
47723   Abilene  United States   32.95N   100.53W 1834-01-01  
137067    Akron  United States   40.99N    80.95W 1743-12-01  
137068    Akron  United States   40.99N    80.95W 1744-01-01  
137069    Akron  United States   40.99N    80.95W 1744-02-01  
137070    Akron  United States   40.99N    80.95W 1744-03-01

In [25]:
# drop all Null And NaN data.
df_drop = df_temperature[df_temperature.AverageTemperature.isnull() & df_temperature.AverageTemperatureUncertainty.isnull()]
df_drop.head()

dt  AverageTemperature  AverageTemperatureUncertainty  \
47723   1834-01-01                 NaN                            NaN   
137067  1743-12-01                 NaN                            NaN   
137068  1744-01-01                 NaN                            NaN   
137069  1744-02-01                 NaN                            NaN   
137070  1744-03-01                 NaN                            NaN   

           City        Country Latitude Longitude  convertDt  
47723   Abilene  United States   32.95N   100.53W 1834-01-01  
137067    Akron  United States   40.99N    80.95W 1743-12-01  
137068    Akron  United States   40.99N    80.95W 1744-01-01  
137069    Akron  United States   40.99N    80.95W 1744-02-01  
137070    Akron  United States   40.99N    80.95W 1744-03-01

In [26]:
df_temperature = df_temperature.drop(df_temperature[df_temperature.AverageTemperature.isnull() & df_temperature.AverageTemperatureUncertainty.isnull()].index, inplace = False)

In [27]:
df_temperature.head()

dt  AverageTemperature  AverageTemperatureUncertainty     City  \
47555  1820-01-01               2.101                          3.217  Abilene   
47556  1820-02-01               6.926                          2.853  Abilene   
47557  1820-03-01              10.767                          2.395  Abilene   
47558  1820-04-01              17.989                          2.202  Abilene   
47559  1820-05-01              21.809                          2.036  Abilene   

             Country Latitude Longitude  convertDt  
47555  United States   32.95N   100.53W 1820-01-01  
47556  United States   32.95N   100.53W 1820-02-01  
47557  United States   32.95N   100.53W 1820-03-01  
47558  United States   32.95N   100.53W 1820-04-01  
47559  United States   32.95N   100.53W 1820-05-01

In [28]:
# Drop duplicate data
duplicateRows = df_temperature[df_temperature.duplicated()]
duplicateRows.shape

(0, 8)

#### Cleaning the Airport Data

In [29]:
airpotcodedf.shape

(55075, 12)

In [30]:
airpotcodedf.head()

ident           type                                name  elevation_ft  \
0   00A       heliport                   Total Rf Heliport          11.0   
1  00AA  small_airport                Aero B Ranch Airport        3435.0   
2  00AK  small_airport                        Lowell Field         450.0   
3  00AL  small_airport                        Epps Airpark         820.0   
4  00AR         closed  Newport Hospital & Clinic Heliport         237.0   

  continent iso_country iso_region  municipality gps_code iata_code  \
0       NaN          US      US-PA      Bensalem      00A       NaN   
1       NaN          US      US-KS         Leoti     00AA       NaN   
2       NaN          US      US-AK  Anchor Point     00AK       NaN   
3       NaN          US      US-AL       Harvest     00AL       NaN   
4       NaN          US      US-AR       Newport      NaN       NaN   

  local_code                            coordinates  
0        00A     -74.93360137939453, 40.07080078125  
1       00AA                 -101.473911, 38.704022  
2       00AK            -151.695999146, 59.94919968  
3       00AL  -86.77030181884766, 34.86479949951172  
4        NaN                    -91.254898, 35.6087

In [31]:
airpotcodedf.isnull().sum()

ident               0
type                0
name                0
elevation_ft     7006
continent       27719
iso_country       247
iso_region          0
municipality     5676
gps_code        14045
iata_code       45886
local_code      26389
coordinates         0
dtype: int64

In [32]:
# Not need to remove null or drop duplicate
duplicateRows = airpotcodedf[airpotcodedf.duplicated()]
duplicateRows.shape

(0, 12)

#### Cleaning the US Cities Demographics

In [33]:
demographicdf.isnull().sum()

City                       0
State                      0
Median Age                 0
Male Population            3
Female Population          3
Total Population           0
Number of Veterans        13
Foreign-born              13
Average Household Size    16
State Code                 0
Race                       0
Count                      0
dtype: int64

In [34]:
demographicdf.shape
demographicdf['City'] = demographicdf['City'].str.strip().str.upper()
demographicdf['State'] = demographicdf['State'].str.strip().str.upper()
demographicdf['Race'] = demographicdf['Race'].str.strip().str.upper()
demographicdf['State Code'] = demographicdf['State Code'].str.strip()

In [35]:
demographicdf.head()

City          State  Median Age  Male Population  \
0     SILVER SPRING       MARYLAND        33.8          40601.0   
1            QUINCY  MASSACHUSETTS        41.0          44129.0   
2            HOOVER        ALABAMA        38.5          38040.0   
3  RANCHO CUCAMONGA     CALIFORNIA        34.5          88127.0   
4            NEWARK     NEW JERSEY        34.6         138040.0   

   Female Population  Total Population  Number of Veterans  Foreign-born  \
0            41862.0             82463              1562.0       30908.0   
1            49500.0             93629              4147.0       32935.0   
2            46799.0             84839              4819.0        8229.0   
3            87105.0            175232              5821.0       33878.0   
4           143873.0            281913              5829.0       86253.0   

   Average Household Size State Code                       Race  Count  
0                    2.60         MD         HISPANIC OR LATINO  25924  
1                    2.39         MA                      WHITE  58723  
2                    2.58         AL                      ASIAN   4759  
3                    3.18         CA  BLACK OR AFRICAN-AMERICAN  24437  
4                    2.73         NJ                      WHITE  76402

In [36]:
# found duplicate Row
duplicateRows = demographicdf[demographicdf.duplicated(['City', 'Race'])]
check_df = demographicdf[(demographicdf['City'] == 'WILMINGTON') & (demographicdf['Race'] == 'ASIAN')]
check_df.head()

City           State  Median Age  Male Population  \
102  WILMINGTON  NORTH CAROLINA        35.5          52346.0   
177  WILMINGTON        DELAWARE        36.4          32680.0   

     Female Population  Total Population  Number of Veterans  Foreign-born  \
102            63601.0            115947              5908.0        7401.0   
177            39277.0             71957              3063.0        3336.0   

     Average Household Size State Code   Race  Count  
102                    2.24         NC  ASIAN   3152  
177                    2.45         DE  ASIAN   1193

In [37]:
# Let's check with City, Race, State and no found duplicate
duplicateRows = demographicdf[demographicdf.duplicated(['City', 'Race', 'State'])]
duplicateRows.head()

Empty DataFrame
Columns: [City, State, Median Age, Male Population, Female Population, Total Population, Number of Veterans, Foreign-born, Average Household Size, State Code, Race, Count]
Index: []

#### Cleaning the Immigration

In [36]:
df_spark.count()

3096313

In [38]:
df_spark.printSchema()

root
 |-- cicid: double (nullable = true)
 |-- i94yr: double (nullable = true)
 |-- i94mon: double (nullable = true)
 |-- i94cit: double (nullable = true)
 |-- i94res: double (nullable = true)
 |-- i94port: string (nullable = true)
 |-- arrdate: double (nullable = true)
 |-- i94mode: double (nullable = true)
 |-- i94addr: string (nullable = true)
 |-- depdate: double (nullable = true)
 |-- i94bir: double (nullable = true)
 |-- i94visa: double (nullable = true)
 |-- count: double (nullable = true)
 |-- dtadfile: string (nullable = true)
 |-- visapost: string (nullable = true)
 |-- occup: string (nullable = true)
 |-- entdepa: string (nullable = true)
 |-- entdepd: string (nullable = true)
 |-- entdepu: string (nullable = true)
 |-- matflag: string (nullable = true)
 |-- biryear: double (nullable = true)
 |-- dtaddto: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- insnum: string (nullable = true)
 |-- airline: string (nullable = true)
 |-- admnum: double (nullable = 

In [39]:
df_spark.show(5)

+-----+------+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------+-----+-------+-------+-------+-------+-------+--------+------+------+-------+--------------+-----+--------+
|cicid| i94yr|i94mon|i94cit|i94res|i94port|arrdate|i94mode|i94addr|depdate|i94bir|i94visa|count|dtadfile|visapost|occup|entdepa|entdepd|entdepu|matflag|biryear| dtaddto|gender|insnum|airline|        admnum|fltno|visatype|
+-----+------+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------+-----+-------+-------+-------+-------+-------+--------+------+------+-------+--------------+-----+--------+
|  6.0|2016.0|   4.0| 692.0| 692.0|    XXX|20573.0|   null|   null|   null|  37.0|    2.0|  1.0|    null|    null| null|      T|   null|      U|   null| 1979.0|10282016|  null|  null|   null| 1.897628485E9| null|      B2|
|  7.0|2016.0|   4.0| 254.0| 276.0|    ATL|20551.0|    1.0|     AL|   null|  25.0|    3.0|  1.0|20130811|     SE

In [40]:
df_spark.createOrReplaceTempView("immigration_table")

### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
Map out the conceptual data model and explain why you chose that model

#### 3.2 Mapping Out Data Pipelines
List the steps necessary to pipeline the data into the chosen data model

In [41]:
# number of cicid is the primary key
spark.sql("""
SELECT COUNT(DISTINCT cicid)
FROM immigration_table
""").show()

+---------------------+
|count(DISTINCT cicid)|
+---------------------+
|              3096313|
+---------------------+



In [44]:
# prepares dicitonaries
# countries:
import csv
  
country_info = ['code', 'country']
country_dict = []
with open("countries.txt") as fp:
    lines = fp.readlines()
    for line in lines:
        (a, b) = line.split('=')
        dict = {}
        dict['code'] = a.replace("'", '').strip()
        dict['country'] = b.replace("'", '').strip()
        country_dict.append(dict)

with open('countries.csv', 'w') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames = country_info,  delimiter=',')
    writer.writeheader()
    writer.writerows(country_dict)
    
# ports:
port_info = ['code', 'location', 'state']
port_dict = []
with open("ports.txt") as fp:
    lines = fp.readlines()
    for line in lines:
        (a, b) = line.split('=')
        dict = {}
        dict['code'] = a.replace("'", '').strip()
        port = b.replace("'", '').strip()
        port_list = port.split(',')
        dict['location'] = port_list[0].strip()
        if len(port_list) > 1:
            dict['state'] = port_list[1].strip()
        elif port_list[0] == 'MARIPOSA AZ':
            dict['location'] = 'MARIPOSA'
            dict['state'] = 'AZ'
        port_dict.append(dict)

with open('ports.csv', 'w') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames = port_info,  delimiter=',')
    writer.writeheader()
    writer.writerows(port_dict)
    
# states:
state_info = ['code', 'state']
state_dict = []
with open("states.txt") as fp:
    lines = fp.readlines()
    for line in lines:
        (a, b) = line.split('=')
        dict = {}
        dict['code'] = a.replace("'", '').strip()
        dict['state'] = b.replace("'", '').strip()
        state_dict.append(dict)

with open('states.csv', 'w') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames = state_info,  delimiter=',')
    writer.writeheader()
    writer.writerows(state_dict)

In [45]:
# number of undefined mode
spark.sql("""
SELECT COUNT(DISTINCT cicid)
FROM immigration_table
WHERE i94mode NOT IN ('1.0', '2.0', '3.0')
""").show()

+---------------------+
|count(DISTINCT cicid)|
+---------------------+
|                 8560|
+---------------------+



In [46]:
# just get the mode in 1 = Air Plane
spark.sql("""
SELECT COUNT(DISTINCT cicid)
FROM immigration_table WHERE i94mode = '1.0'
""").show()

+---------------------+
|count(DISTINCT cicid)|
+---------------------+
|              2994505|
+---------------------+



In [47]:
# check the gender
spark.sql("""
SELECT DISTINCT gender
    FROM immigration_table
""").show()

# remove all gender that not undefine
spark.sql("""
SELECT COUNt(cicid)
    FROM immigration_table 
    WHERE gender = 'U'
""").show()

spark.sql("""
SELECT COUNt(cicid) as X_count
    FROM immigration_table 
    WHERE gender = 'X'
""").show()

spark.sql("""
SELECT COUNt(cicid) as Null_count
    FROM immigration_table 
    WHERE gender IS null
""").show()

+------+
|gender|
+------+
|     F|
|  null|
|     M|
|     U|
|     X|
+------+

+------------+
|count(cicid)|
+------------+
|         467|
+------------+

+-------+
|X_count|
+-------+
|   1610|
+-------+

+----------+
|Null_count|
+----------+
|    414269|
+----------+



In [48]:
spark.sql("""
SELECT * 
FROM immigration_table LIMIT 5
""").show()

+-----+------+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------+-----+-------+-------+-------+-------+-------+--------+------+------+-------+--------------+-----+--------+
|cicid| i94yr|i94mon|i94cit|i94res|i94port|arrdate|i94mode|i94addr|depdate|i94bir|i94visa|count|dtadfile|visapost|occup|entdepa|entdepd|entdepu|matflag|biryear| dtaddto|gender|insnum|airline|        admnum|fltno|visatype|
+-----+------+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------+-----+-------+-------+-------+-------+-------+--------+------+------+-------+--------------+-----+--------+
|  6.0|2016.0|   4.0| 692.0| 692.0|    XXX|20573.0|   null|   null|   null|  37.0|    2.0|  1.0|    null|    null| null|      T|   null|      U|   null| 1979.0|10282016|  null|  null|   null| 1.897628485E9| null|      B2|
|  7.0|2016.0|   4.0| 254.0| 276.0|    ATL|20551.0|    1.0|     AL|   null|  25.0|    3.0|  1.0|20130811|     SE

In [50]:
df_portCodes = pd.read_csv('ports.csv')

In [51]:
df_portCodesNotQualified = df_portCodes[df_portCodes['state'].isna()]
df_portCodesNotQualified

code                       location state
76   WAS                  WASHINGTON DC   NaN
516  XXX           NOT REPORTED/UNKNOWN   NaN
517  888      UNIDENTIFED AIR / SEAPORT   NaN
518  UNK                    UNKNOWN POE   NaN
575  ZZZ  MEXICO Land (Banco de Mexico)   NaN
576  CHN             No PORT Code (CHN)   NaN
578  MAA                      Abu Dhabi   NaN
591  FRG          Collapsed (FOK) 06/15   NaN
592  HRL          Collapsed (HLG) 06/15   NaN
593  ISP          Collapsed (FOK) 06/15   NaN
594  JSJ          Collapsed (SAJ) 06/15   NaN
595  BUS          Collapsed (BUF) 06/15   NaN
596  IAG          Collapsed (NIA) 06/15   NaN
597  PHN          Collapsed (PHU) 06/15   NaN
598  STN          Collapsed (STR) 06/15   NaN
599  VMB          Collapsed (VNB) 06/15   NaN
600  T01          Collapsed (SEA) 06/15   NaN
601  PHF             No PORT Code (PHF)   NaN
602  DRV             No PORT Code (DRV)   NaN
603  FTB             No PORT Code (FTB)   NaN
604  GAC             No PORT Code (GAC)   NaN
605  GMT             No PORT Code (GMT)   NaN
606  JFA             No PORT Code (JFA)   NaN
607  JMZ             No PORT Code (JMZ)   NaN
608  NC8             No PORT Code (NC8)   NaN
609  NYL             No PORT Code (NYL)   NaN
610  OAI             No PORT Code (OAI)   NaN
611  PCW             No PORT Code (PCW)   NaN
612  WA5             No PORT Code (WAS)   NaN
613  WTR             No PORT Code (WTR)   NaN
..   ...                            ...   ...
630  SCH             No PORT Code (SCH)   NaN
631  ASI             No PORT Code (ASI)   NaN
632  BKF             No PORT Code (BKF)   NaN
633  DAY             No PORT Code (DAY)   NaN
634  Y62             No PORT Code (Y62)   NaN
635   AG              No PORT Code (AG)   NaN
636  BCM             No PORT Code (BCM)   NaN
637  DEC             No PORT Code (DEC)   NaN
638  PLB             No PORT Code (PLB)   NaN
639  CXO             No PORT Code (CXO)   NaN
640  JBQ             No PORT Code (JBQ)   NaN
641  JIG             No PORT Code (JIG)   NaN
642  OGS             No PORT Code (OGS)   NaN
643  TIW             No PORT Code (TIW)   NaN
644  OTS             No PORT Code (OTS)   NaN
645  AMT             No PORT Code (AMT)   NaN
646  EGE             No PORT Code (EGE)   NaN
647  GPI             No PORT Code (GPI)   NaN
648  NGL             No PORT Code (NGL)   NaN
649  OLM             No PORT Code (OLM)   NaN
650  .GA             No PORT Code (.GA)   NaN
651  CLX             No PORT Code (CLX)   NaN
652   CP              No PORT Code (CP)   NaN
653  FSC             No PORT Code (FSC)   NaN
654   NK              No PORT Code (NK)   NaN
655  ADU             No PORT Code (ADU)   NaN
656  AKT             No PORT Code (AKT)   NaN
657  LIT             No PORT Code (LIT)   NaN
658  A2A             No PORT Code (A2A)   NaN
659  OSN             No PORT Code (OSN)   NaN

[76 rows x 3 columns]

In [52]:
# remove not qualified portCode
df_portCodes = df_portCodes.drop(df_portCodes[df_portCodes['state'].isna()].index, inplace = False)
df_portCodes

code                     location          state
0    ALC                        ALCAN             AK
1    ANC                    ANCHORAGE             AK
2    BAR     BAKER AAF - BAKER ISLAND             AK
3    DAC                DALTONS CACHE             AK
4    PIZ       DEW STATION PT LAY DEW             AK
5    DTH                 DUTCH HARBOR             AK
6    EGL                        EAGLE             AK
7    FRB                    FAIRBANKS             AK
8    HOM                        HOMER             AK
9    HYD                        HYDER             AK
10   JUN                       JUNEAU             AK
11   5KE                    KETCHIKAN             AK
12   KET                    KETCHIKAN             AK
13   MOS     MOSES POINT INTERMEDIATE             AK
14   NIK                      NIKISKI             AK
15   NOM                          NOM             AK
16   PKC                  POKER CREEK             AK
17   ORI               PORT LIONS SPB             AK
18   SKA                      SKAGWAY             AK
19   SNP              ST. PAUL ISLAND             AK
20   TKI                       TOKEEN             AK
21   WRA                     WRANGELL             AK
22   HSV  MADISON COUNTY - HUNTSVILLE             AL
23   MOB                       MOBILE             AL
24   LIA                  LITTLE ROCK       AR (BPS)
25   ROG                  ROGERS ARPT             AR
26   DOU                      DOUGLAS             AZ
27   LUK                    LUKEVILLE             AZ
28   MAP                     MARIPOSA             AZ
29   NAC                         NACO             AZ
..   ...                          ...            ...
558  NBO                      NAIROBI          KENYA
559  NAS                       NASSAU        BAHAMAS
560  NCA                 NORTH CAICOS  TURK & CAIMAN
561  PTY                OMAR TORRIJOS         PANAMA
562  SPV                        PAPUA     NEW GUINEA
563  UIO        QUITO (MARISCAL SUCR)        ECUADOR
564  RIT                         ROME          ITALY
565  SNO           SAKON NAKHON #ARPT       THAILAND
566  SLP        SAN LUIS POTOSI #ARPT         MEXICO
567  SAN                 SAN SALVADOR    EL SALVADOR
568  SRO          SANTANA RAMOS #ARPT       COLOMBIA
569  GRU               GUARULHOS INTL      SAO PAULO
570  SHA                      SHANNON        IRELAND
571  HIL                     SHILLAVO       ETHIOPIA
572  TOK               TOROKINA #ARPT          PAPUA
573  VER                     VERACRUZ         MEXICO
574  LGW                  WEST SUSSEX        ENGLAND
577  CNC               CANNON CORNERS             NY
579  AG0                     MAGNOLIA             AR
580  BHM                   BAR HARBOR             ME
581  BHX                   BIRMINGHAM             AL
582  CAK                        AKRON             OH
583  FOK               SUFFOLK COUNTY             NY
584  LND                       LANDER             WY
585  MAR                        MARFA             TX
586  MLI                       MOLINE             IL
587  RIV                    RIVERSIDE             CA
588  RME                         ROME             NY
589  VNY                     VAN NUYS             CA
590  YUM                         YUMA             AZ

[584 rows x 3 columns]

In [53]:
# nonUSstates
nonUSstates = ['CANADA', 'Canada', 'NETHERLANDS', 'NETH ANTILLES', 'THAILAND', 'ETHIOPIA', 'PRC', 'BERMUDA', 'COLOMBIA', 'ARGENTINA', 'MEXICO', 
               'BRAZIL', 'URUGUAY', 'IRELAND', 'GABON', 'BAHAMAS', 'MX', 'CAYMAN ISLAND', 'SEOUL KOREA', 'JAPAN', 'ROMANIA', 'INDONESIA',
               'SOUTH AFRICA', 'ENGLAND', 'KENYA', 'TURK & CAIMAN', 'PANAMA', 'NEW GUINEA', 'ECUADOR', 'ITALY', 'EL SALVADOR', 'EL DORADO #ARPT', 
               'BA #ARPT' , 'MIGUEL HIDAL' , 'MEXICO CITY' , 'PAPUA' , 'SAO PAULO' , 'MINISTRO PIST']
df_portCodesNotUs = df_portCodes[df_portCodes['state'].isin(nonUSstates)]
df_portCodesNotUs

# remove all non US States
df_portCodes = df_portCodes[~df_portCodes['state'].isin(nonUSstates)]
df_portCodes

code                     location      state
0    ALC                        ALCAN         AK
1    ANC                    ANCHORAGE         AK
2    BAR     BAKER AAF - BAKER ISLAND         AK
3    DAC                DALTONS CACHE         AK
4    PIZ       DEW STATION PT LAY DEW         AK
5    DTH                 DUTCH HARBOR         AK
6    EGL                        EAGLE         AK
7    FRB                    FAIRBANKS         AK
8    HOM                        HOMER         AK
9    HYD                        HYDER         AK
10   JUN                       JUNEAU         AK
11   5KE                    KETCHIKAN         AK
12   KET                    KETCHIKAN         AK
13   MOS     MOSES POINT INTERMEDIATE         AK
14   NIK                      NIKISKI         AK
15   NOM                          NOM         AK
16   PKC                  POKER CREEK         AK
17   ORI               PORT LIONS SPB         AK
18   SKA                      SKAGWAY         AK
19   SNP              ST. PAUL ISLAND         AK
20   TKI                       TOKEEN         AK
21   WRA                     WRANGELL         AK
22   HSV  MADISON COUNTY - HUNTSVILLE         AL
23   MOB                       MOBILE         AL
24   LIA                  LITTLE ROCK   AR (BPS)
25   ROG                  ROGERS ARPT         AR
26   DOU                      DOUGLAS         AZ
27   LUK                    LUKEVILLE         AZ
28   MAP                     MARIPOSA         AZ
29   NAC                         NACO         AZ
..   ...                          ...        ...
501  SPO                      SPOKANE         WA
502  SUM                        SUMAS         WA
503  TAC                       TACOMA         WA
504  PSC           TRI-CITIES - PASCO         WA
505  VAN                    VANCOUVER         WA
506  AGM                       ALGOMA         WI
507  BAY                     BAYFIELD         WI
508  GRB                    GREEN BAY         WI
509  MNW                    MANITOWOC         WI
510  MIL                    MILWAUKEE         WI
511  MSN    TRUAX FIELD - DANE COUNTY         WI
512  CHS                   CHARLESTON         WV
513  CLK                   CLARKSBURG         WV
514  BLF                MERCER COUNTY         WV
515  CSP                       CASPER         WY
522  HAL                      Halifax         NS
544  MTY              GEN M. ESCOBEDO  Monterrey
577  CNC               CANNON CORNERS         NY
579  AG0                     MAGNOLIA         AR
580  BHM                   BAR HARBOR         ME
581  BHX                   BIRMINGHAM         AL
582  CAK                        AKRON         OH
583  FOK               SUFFOLK COUNTY         NY
584  LND                       LANDER         WY
585  MAR                        MARFA         TX
586  MLI                       MOLINE         IL
587  RIV                    RIVERSIDE         CA
588  RME                         ROME         NY
589  VNY                     VAN NUYS         CA
590  YUM                         YUMA         AZ

[530 rows x 3 columns]

In [54]:
# clean df airports
df_airports = airpotcodedf[airpotcodedf['iso_country'].fillna('').str.upper().str.contains('US')].copy()
df_airports.head()

ident           type                                name  elevation_ft  \
0   00A       heliport                   Total Rf Heliport          11.0   
1  00AA  small_airport                Aero B Ranch Airport        3435.0   
2  00AK  small_airport                        Lowell Field         450.0   
3  00AL  small_airport                        Epps Airpark         820.0   
4  00AR         closed  Newport Hospital & Clinic Heliport         237.0   

  continent iso_country iso_region  municipality gps_code iata_code  \
0       NaN          US      US-PA      Bensalem      00A       NaN   
1       NaN          US      US-KS         Leoti     00AA       NaN   
2       NaN          US      US-AK  Anchor Point     00AK       NaN   
3       NaN          US      US-AL       Harvest     00AL       NaN   
4       NaN          US      US-AR       Newport      NaN       NaN   

  local_code                            coordinates  
0        00A     -74.93360137939453, 40.07080078125  
1       00AA                 -101.473911, 38.704022  
2       00AK            -151.695999146, 59.94919968  
3       00AL  -86.77030181884766, 34.86479949951172  
4        NaN                    -91.254898, 35.6087

In [55]:
df_airports['type'].unique()

array(['heliport', 'small_airport', 'closed', 'seaplane_base',
       'balloonport', 'medium_airport', 'large_airport'], dtype=object)

In [56]:
nonQualifiedValues = [ 'heliport', 'closed', 'seaplane_base', 'balloonport' ]
df_airports = df_airports[~df_airports['type'].str.strip().isin(nonQualifiedValues)].copy()
df_airports = df_airports[~df_airports['municipality'].isna()].copy()
df_airports['ident'] = df_airports['ident'].str.strip()
df_airports['municipality'] = df_airports['municipality'].str.strip().str.upper()
df_airports.head()

ident           type                  name  elevation_ft continent  \
1  00AA  small_airport  Aero B Ranch Airport        3435.0       NaN   
2  00AK  small_airport          Lowell Field         450.0       NaN   
3  00AL  small_airport          Epps Airpark         820.0       NaN   
5  00AS  small_airport        Fulton Airport        1100.0       NaN   
6  00AZ  small_airport        Cordes Airport        3810.0       NaN   

  iso_country iso_region  municipality gps_code iata_code local_code  \
1          US      US-KS         LEOTI     00AA       NaN       00AA   
2          US      US-AK  ANCHOR POINT     00AK       NaN       00AK   
3          US      US-AL       HARVEST     00AL       NaN       00AL   
5          US      US-OK          ALEX     00AS       NaN       00AS   
6          US      US-AZ        CORDES     00AZ       NaN       00AZ   

                               coordinates  
1                   -101.473911, 38.704022  
2              -151.695999146, 59.94919968  
3    -86.77030181884766, 34.86479949951172  
5                  -97.8180194, 34.9428028  
6  -112.16500091552734, 34.305599212646484

In [57]:
df_airports['iata_code'].unique()
df_airports['iata_code'] = df_airports['iata_code'].fillna('').astype(str)
df_airports['gps_code'] = df_airports['gps_code'].fillna('').astype(str)
df_airports['local_code'] = df_airports['local_code'].fillna('').astype(str)

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.

In [58]:
# Staging data
df_countryCodes = pd.read_csv('countries.csv')
df_portCodes = pd.read_csv('ports.csv')

In [59]:
# Transform data and convert dictionaries to views in spark context
spark_df_countryCodes = spark.createDataFrame(df_countryCodes)
spark_df_countryCodes .createOrReplaceTempView("countries")

In [60]:
# Check portcodes
df_portCodesNotQualified = df_portCodes[df_portCodes['state'].isna()]

# remove not qualified portCode
df_portCodes = df_portCodes.drop(df_portCodes[df_portCodes['state'].isna()].index, inplace = False)

# nonUSstates
nonUSstates = ['CANADA', 'Canada', 'NETHERLANDS', 'NETH ANTILLES', 'THAILAND', 'ETHIOPIA', 'PRC', 'BERMUDA', 'COLOMBIA', 'ARGENTINA', 'MEXICO', 
               'BRAZIL', 'URUGUAY', 'IRELAND', 'GABON', 'BAHAMAS', 'MX', 'CAYMAN ISLAND', 'SEOUL KOREA', 'JAPAN', 'ROMANIA', 'INDONESIA',
               'SOUTH AFRICA', 'ENGLAND', 'KENYA', 'TURK & CAIMAN', 'PANAMA', 'NEW GUINEA', 'ECUADOR', 'ITALY', 'EL SALVADOR', 'EL DORADO #ARPT', 
               'BA #ARPT' , 'MIGUEL HIDAL' , 'MEXICO CITY' , 'PAPUA' , 'SAO PAULO' , 'MINISTRO PIST']
df_portCodesNotUs = df_portCodes[df_portCodes['state'].isin(nonUSstates)]

# remove all non US States
df_portCodes = df_portCodes[~df_portCodes['state'].isin(nonUSstates)]
df_portCodes

code                     location      state
0    ALC                        ALCAN         AK
1    ANC                    ANCHORAGE         AK
2    BAR     BAKER AAF - BAKER ISLAND         AK
3    DAC                DALTONS CACHE         AK
4    PIZ       DEW STATION PT LAY DEW         AK
5    DTH                 DUTCH HARBOR         AK
6    EGL                        EAGLE         AK
7    FRB                    FAIRBANKS         AK
8    HOM                        HOMER         AK
9    HYD                        HYDER         AK
10   JUN                       JUNEAU         AK
11   5KE                    KETCHIKAN         AK
12   KET                    KETCHIKAN         AK
13   MOS     MOSES POINT INTERMEDIATE         AK
14   NIK                      NIKISKI         AK
15   NOM                          NOM         AK
16   PKC                  POKER CREEK         AK
17   ORI               PORT LIONS SPB         AK
18   SKA                      SKAGWAY         AK
19   SNP              ST. PAUL ISLAND         AK
20   TKI                       TOKEEN         AK
21   WRA                     WRANGELL         AK
22   HSV  MADISON COUNTY - HUNTSVILLE         AL
23   MOB                       MOBILE         AL
24   LIA                  LITTLE ROCK   AR (BPS)
25   ROG                  ROGERS ARPT         AR
26   DOU                      DOUGLAS         AZ
27   LUK                    LUKEVILLE         AZ
28   MAP                     MARIPOSA         AZ
29   NAC                         NACO         AZ
..   ...                          ...        ...
501  SPO                      SPOKANE         WA
502  SUM                        SUMAS         WA
503  TAC                       TACOMA         WA
504  PSC           TRI-CITIES - PASCO         WA
505  VAN                    VANCOUVER         WA
506  AGM                       ALGOMA         WI
507  BAY                     BAYFIELD         WI
508  GRB                    GREEN BAY         WI
509  MNW                    MANITOWOC         WI
510  MIL                    MILWAUKEE         WI
511  MSN    TRUAX FIELD - DANE COUNTY         WI
512  CHS                   CHARLESTON         WV
513  CLK                   CLARKSBURG         WV
514  BLF                MERCER COUNTY         WV
515  CSP                       CASPER         WY
522  HAL                      Halifax         NS
544  MTY              GEN M. ESCOBEDO  Monterrey
577  CNC               CANNON CORNERS         NY
579  AG0                     MAGNOLIA         AR
580  BHM                   BAR HARBOR         ME
581  BHX                   BIRMINGHAM         AL
582  CAK                        AKRON         OH
583  FOK               SUFFOLK COUNTY         NY
584  LND                       LANDER         WY
585  MAR                        MARFA         TX
586  MLI                       MOLINE         IL
587  RIV                    RIVERSIDE         CA
588  RME                         ROME         NY
589  VNY                     VAN NUYS         CA
590  YUM                         YUMA         AZ

[530 rows x 3 columns]

In [61]:
spark_df_portCodes = spark.createDataFrame(df_portCodes)
spark_df_portCodes .createOrReplaceTempView("portCodes")

In [62]:
# Use an inner join to drop invalid codes country of citizenship
spark.sql("""
SELECT i.*, c.country AS citizenship_country
    FROM immigration_table i
    INNER JOIN countries c
    ON i.i94cit = c.code
""").createOrReplaceTempView("immigration_table")

# Use an inner join to drop invalid codes country of residence
spark.sql("""
SELECT i.*, c.country AS residence_country
    FROM immigration_table i
    INNER JOIN countries c
    ON i.i94res = c.code
""").createOrReplaceTempView("immigration_table")

# People of indeterminate gender are few, so can remove them.
spark.sql("""
SELECT *
    FROM immigration_table 
    WHERE gender IN ('M', 'F')
""").createOrReplaceTempView("immigration_table")

# Specifically, SAS stores dates as numeric values equal to the number of days from January 1, 1960
spark.sql("""
SELECT *, 
    date_add(to_date('1960-01-01'), arrdate) AS arrival_date,
    date_add(to_date('1960-01-01'), depdate) AS departure_date,
    (2016-biryear) AS age
FROM immigration_table
""").createOrReplaceTempView("immigration_table")

# Add entry_port names and entry port states to immigration_table
spark.sql("""
SELECT im.*, 
    pc.location AS entry_port, 
    pc.state AS entry_port_state
FROM immigration_table im 
INNER JOIN portCodes pc
ON im.i94port = pc.code
""").createOrReplaceTempView("immigration_table")

# convert i94visa -> visa_type
spark.sql("""
SELECT *, CASE 
    WHEN i94visa = '1.0' THEN 'Business'
    WHEN i94visa = '2.0' THEN 'Pleasure'
    WHEN i94visa = '3.0' THEN 'Student'
    ELSE 'N/A' END AS visa_type 
    FROM immigration_table 
""").createOrReplaceTempView("immigration_table")

# convert I94MODE -> mode_type
spark.sql("""
SELECT *, CASE 
    WHEN i94mode = '1.0' THEN 'Air'
    WHEN i94mode = '2.0' THEN 'Sea'
    WHEN i94mode = '3.0' THEN 'Land'
    WHEN i94mode = '9.0' THEN 'Not reported'
    ELSE 'N/A' END AS mode_type 
    FROM immigration_table 
""").createOrReplaceTempView("immigration_table")

# just get the mode in 1. Remove all entries into US not in Air travel
spark.sql("""
SELECT *
    FROM immigration_table 
    WHERE i94mode = '1.0'
""").createOrReplaceTempView("immigration_table")

In [63]:
# Insert fact table into spark dataframe
fact_immigration = spark.sql("""
SELECT 
    cicid, 
    citizenship_country, 
    residence_country,
    UPPER (entry_port) AS city,
    UPPER (entry_port_state) AS state,
    arrival_date,
    departure_date,
    age,
    visa_type,
    visatype AS detailed_visa_type,
    i94yr,
    i94mon,
    i94addr,
    i94bir,
    count,
    mode_type,
    gender,
    insnum,
    airline,
    admnum,
    fltno
FROM immigration_table    
""")

In [64]:
# Create dimension table time
dim_time = spark.sql("""
SELECT 
    DISTINCT arrival_date AS date
    FROM immigration_table
WHERE arrival_date IS NOT NULL
UNION
SELECT 
    DISTINCT departure_date AS date
    FROM immigration_table
WHERE departure_date IS NOT NULL
""")
dim_time.createOrReplaceTempView("dim_time_table")

In [65]:
# Extract year, month, day, weekofyear, dayofweek, dayofyear from the date
dim_time = spark.sql("""
SELECT 
    date, 
    YEAR(date) AS year, 
    MONTH(date) AS month, 
    DAY(date) AS day, 
    WEEKOFYEAR(date) AS week, 
    DAYOFWEEK(date) as weekday, 
    DAYOFYEAR(date) yearday
FROM dim_time_table
ORDER BY date ASC
""")

In [66]:
spark_df_temperature = spark.createDataFrame(df_temperature)
spark_df_temperature .createOrReplaceTempView("temperature")

In [67]:
dim_temperature = spark.sql("""
SELECT
    DISTINCT dt, city,
    AVG(AverageTemperature) OVER (PARTITION BY dt, City) AS average_temperature, 
    AVG(AverageTemperatureUncertainty)  OVER (PARTITION BY dt, City) AS average_termperature_uncertainty
FROM temperature
""")

In [68]:
spark_df_demographics = spark.createDataFrame(demographicdf)
spark_df_demographics.createOrReplaceTempView("demographics")

In [69]:
dim_demographics = spark.sql("""
SELECT  
    City as city,
    State as state, 
    `Median Age` AS median_age, 
    `Male Population` AS male_population, 
    `Female Population` AS female_population, 
    `Total Population` AS total_population, 
    `Foreign-born` AS foreign_born, 
    `Average Household Size` AS average_household_size,
    `State Code` AS state_code, 
    Race as race, 
    Count as count
FROM demographics
""")

In [70]:
# clean df airports
df_airports = airpotcodedf[airpotcodedf['iso_country'].fillna('').str.upper().str.contains('US')].copy()
df_airports['type'].unique()

array(['heliport', 'small_airport', 'closed', 'seaplane_base',
       'balloonport', 'medium_airport', 'large_airport'], dtype=object)

In [71]:
nonQualifiedValues = [ 'heliport', 'closed', 'seaplane_base', 'balloonport' ]
df_airports = df_airports[~df_airports['type'].str.strip().isin(nonQualifiedValues)].copy()
df_airports = df_airports[~df_airports['municipality'].isna()].copy()
df_airports['ident'] = df_airports['ident'].str.strip()
df_airports['municipality'] = df_airports['municipality'].str.strip().str.upper()
df_airports.head()

ident           type                  name  elevation_ft continent  \
1  00AA  small_airport  Aero B Ranch Airport        3435.0       NaN   
2  00AK  small_airport          Lowell Field         450.0       NaN   
3  00AL  small_airport          Epps Airpark         820.0       NaN   
5  00AS  small_airport        Fulton Airport        1100.0       NaN   
6  00AZ  small_airport        Cordes Airport        3810.0       NaN   

  iso_country iso_region  municipality gps_code iata_code local_code  \
1          US      US-KS         LEOTI     00AA       NaN       00AA   
2          US      US-AK  ANCHOR POINT     00AK       NaN       00AK   
3          US      US-AL       HARVEST     00AL       NaN       00AL   
5          US      US-OK          ALEX     00AS       NaN       00AS   
6          US      US-AZ        CORDES     00AZ       NaN       00AZ   

                               coordinates  
1                   -101.473911, 38.704022  
2              -151.695999146, 59.94919968  
3    -86.77030181884766, 34.86479949951172  
5                  -97.8180194, 34.9428028  
6  -112.16500091552734, 34.305599212646484

In [72]:
df_airports['iata_code'] = df_airports['iata_code'].fillna('').astype(str)
df_airports['gps_code'] = df_airports['gps_code'].fillna('').astype(str)
df_airports['local_code'] = df_airports['local_code'].fillna('').astype(str)

In [73]:
spark_df_airport = spark.createDataFrame(df_airports)
spark_df_airport.createOrReplaceTempView("airports")

In [74]:
dim_airports = spark.sql("""
SELECT 
    ident, 
    type, 
    name, 
    elevation_ft, 
    iso_country, 
    iso_region, 
    municipality, 
    iata_code, 
    gps_code, 
    local_code 
FROM airports
""")

In [76]:
dim_demographics.createOrReplaceTempView("dim_demographics")
dim_time.createOrReplaceTempView("dim_time")
dim_airports.createOrReplaceTempView("dim_airports")
dim_temperature.createOrReplaceTempView("dim_temperature")
fact_immigration.createOrReplaceTempView("fact_immigration")

#### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness
 
Run Quality Checks

In [77]:
# Perform quality checks here
def quality_check(fact_immigration: DataFrame, 
                  dim_demographics: DataFrame, 
                  dim_time: DataFrame, 
                  dim_airports: DataFrame, 
                  dim_temperature: DataFrame, sparkContext) -> None:
    """
    Performs two data quality checks against dataframes.
    1. Each table is not empty.
    2. Validate that immigrations fact table contains only valid values as per dimensional tables.
    """
    error_count = 0
    if fact_immigration.count() == 0:
        error_count += 1
        print("Invalid dataset. Immigrations fact table is empty.")
    if dim_demographics.count() == 0:
        error_count += 1
        print("Invalid dataset. Demographics dim table is empty.")
    if dim_time.count() == 0:
        error_count += 1
        print("Invalid dataset. Time dim table is empty.")
    if dim_airports.count() == 0:
        error_count += 1
        print("Invalid dataset. Airports dim table is empty.")
    if dim_temperature.count() == 0:
        error_count += 1
        print("Invalid dataset. Temperature dim table is empty.")
    
    returnedImmi = sparkContext.sql(f"""SELECT COUNT(*) as nbr FROM immigration_table WHERE cicid IS NULL""")
    if returnedImmi.head()[0] > 0:
        error_count += 1
        print(f"Data quality check failed! Found NULL values in cicid column!")
    
    returnedDemo = sparkContext.sql(f"""SELECT COUNT(*) as nbr FROM demographics WHERE `State Code` IS NULL OR City IS NULL """)
    if returnedDemo.head()[0] > 0:
        error_count += 1
        print(f"Data quality check failed! Found NULL values in state_code, city column!")
    
    returnedTime = sparkContext.sql(f"""SELECT COUNT(*) as nbr FROM dim_time_table WHERE date IS NULL""")
    if returnedTime.head()[0] > 0:
        error_count += 1
        print(f"Data quality check failed! Found NULL values in date column!")
    
    returnedAirp = sparkContext.sql(f"""SELECT COUNT(*) as nbr FROM airports WHERE ident IS NULL""")
    if returnedAirp.head()[0] > 0:
        error_count += 1
        print(f"Data quality check failed! Found NULL values in ident column!")
    
    returnedTemp = sparkContext.sql(f"""SELECT COUNT(*) as nbr FROM temperature WHERE dt IS NULL OR city IS NULL """)
    if returnedTemp.head()[0] > 0:
        error_count += 1
        print(f"Data quality check failed! Found NULL values in dt, city column!")
    
    if error_count == 0:
        print(f"All tables passed.")
    else:
        print(f"One or more tables failed.")

In [78]:
# Check qualify values
quality_check(fact_immigration, dim_demographics, dim_time, dim_airports, dim_temperature, spark)

All tables passed.


#### 4.3 Data dictionary 
Create a data dictionary for your data model. For each field, provide a brief description of what the data is and where it came from. You can include the data dictionary in the notebook or in a separate file.

#### Step 5: Complete Project Write Up
* Clearly state the rationale for the choice of tools and technologies for the project.
* Propose how often the data should be updated and why.
* Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x.
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
 * The database needed to be accessed by 100+ people.

When data increased by 100x: Our data would bs store in AWS S3. Still use Spark as it as our data processing platform since it is the best suited for large datasets

When need to scheduled data pipelines, We can use Apache Airflow to perform ETL and data quality validation

When database need to access by 100+ people: We can use postgres database on a redshift cluster